In [1]:
import json
from pprint import pprint

from torch.nn import Conv2d, MaxPool2d, AdaptiveAvgPool2d, ReLU, Sequential

from lib.fully_connected_net import FullyConnectedNet
from lib.flatten import Flatten


In [2]:
fname = 'DNNs/alexnet_2_65_120190602141824593525_created/k_4/model_params.json'

In [3]:
with open(fname) as json_file:
    layers = json.load(json_file)
    
pprint(layers)

[{'in_channels': 1,
  'kernel_height': 1,
  'kernel_width': 5,
  'out_channels': 132,
  'padding_height': 3,
  'padding_width': 0,
  'stride_height': 4,
  'stride_width': 3,
  'type': 'conv2d'},
 {'kernel_height': 1,
  'kernel_width': 10,
  'stride_height': 2,
  'stride_width': 3,
  'type': 'maxpool2d'},
 {'in_channels': 132,
  'kernel_height': 1,
  'kernel_width': 1,
  'out_channels': 145,
  'padding_height': 0,
  'padding_width': 0,
  'stride_height': 1,
  'stride_width': 3,
  'type': 'conv2d'},
 {'kernel_height': 1,
  'kernel_width': 1,
  'stride_height': 2,
  'stride_width': 4,
  'type': 'maxpool2d'},
 {'in_channels': 145,
  'kernel_height': 2,
  'kernel_width': 2,
  'out_channels': 165,
  'padding_height': 1,
  'padding_width': 2,
  'stride_height': 3,
  'stride_width': 1,
  'type': 'conv2d'},
 {'in_channels': 165,
  'kernel_height': 3,
  'kernel_width': 7,
  'out_channels': 28,
  'padding_height': 2,
  'padding_width': 3,
  'stride_height': 2,
  'stride_width': 1,
  'type': 'conv

In [4]:
modules = []

for layer in layers:
    if layer['type'] == 'conv2d':
        module = Conv2d(layer['in_channels'],
                        layer['out_channels'],
                        (layer['kernel_height'], layer['kernel_width']),
                        stride=(layer['stride_height'], layer['stride_width'])
                       )
    if layer['type'] == 'maxpool2d':
        module = MaxPool2d((layer['kernel_height'], layer['kernel_width']),
                           stride=(layer['stride_height'], layer['stride_width']),
                          )
    
    if layer['type'] == 'adaptiveavgpool2d':
        module = AdaptiveAvgPool2d((layer['out_height'], layer['out_width']))
        
    if layer['type'] == 'fcs':
        flatten = Flatten()
        modules.append(flatten)
        fcs_dropout = 0
        batch_norm = True
        module = FullyConnectedNet(layer['input_size'], layer['output_size'], fcs_dropout, \
                                   batch_norm, layer['width'], layer['num_layers'])
        
        
    modules.append(module)
    
    if layer['type'] == 'conv2d':
        modules.append(ReLU(inplace=True))
        
    
    

In [5]:
help(FullyConnectedNet)

Help on class FullyConnectedNet in module lib.fully_connected_net:

class FullyConnectedNet(torch.nn.modules.module.Module)
 |  FullyConnectedNet(fcs_input_size, output_size, fcs_dropout, batch_norm, fcs_hidden_size, fcs_num_hidden_layers)
 |  
 |  Fully connected network. ReLU is the activation function.
 |      Network parameters are intialized with a normal distribution.
 |  Args:
 |      input_size
 |      output_size
 |      fcs_num_hidden_layers
 |      num_hidden_layers
 |      dropout
 |      dropout_input
 |  
 |  Method resolution order:
 |      FullyConnectedNet
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, fcs_input_size, output_size, fcs_dropout, batch_norm, fcs_hidden_size, fcs_num_hidden_layers)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  forward(self, x)
 |      Defines the computation performed at every call.
 |      
 |      Should be overridden by all subclasse

In [6]:
modules

[Conv2d(1, 132, kernel_size=(1, 5), stride=(4, 3)),
 ReLU(inplace),
 MaxPool2d(kernel_size=(1, 10), stride=(2, 3), padding=0, dilation=1, ceil_mode=False),
 Conv2d(132, 145, kernel_size=(1, 1), stride=(1, 3)),
 ReLU(inplace),
 MaxPool2d(kernel_size=(1, 1), stride=(2, 4), padding=0, dilation=1, ceil_mode=False),
 Conv2d(145, 165, kernel_size=(2, 2), stride=(3, 1)),
 ReLU(inplace),
 Conv2d(165, 28, kernel_size=(3, 7), stride=(2, 1)),
 ReLU(inplace),
 Conv2d(28, 91, kernel_size=(2, 6), stride=(1, 1)),
 ReLU(inplace),
 MaxPool2d(kernel_size=(1, 1), stride=(3, 3), padding=0, dilation=1, ceil_mode=False),
 AdaptiveAvgPool2d(output_size=(1, 1)),
 Flatten(),
 FullyConnectedNet(
   (layers): ModuleList(
     (0): Linear(in_features=91, out_features=128, bias=True)
     (1): Linear(in_features=128, out_features=130, bias=True)
   )
   (batch_norm_layers): ModuleList(
     (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (relu): ReLU()
   (dropout): Dr

In [7]:
Sequential(*modules)

Sequential(
  (0): Conv2d(1, 132, kernel_size=(1, 5), stride=(4, 3))
  (1): ReLU(inplace)
  (2): MaxPool2d(kernel_size=(1, 10), stride=(2, 3), padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(132, 145, kernel_size=(1, 1), stride=(1, 3))
  (4): ReLU(inplace)
  (5): MaxPool2d(kernel_size=(1, 1), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(145, 165, kernel_size=(2, 2), stride=(3, 1))
  (7): ReLU(inplace)
  (8): Conv2d(165, 28, kernel_size=(3, 7), stride=(2, 1))
  (9): ReLU(inplace)
  (10): Conv2d(28, 91, kernel_size=(2, 6), stride=(1, 1))
  (11): ReLU(inplace)
  (12): MaxPool2d(kernel_size=(1, 1), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (13): AdaptiveAvgPool2d(output_size=(1, 1))
  (14): Flatten()
  (15): FullyConnectedNet(
    (layers): ModuleList(
      (0): Linear(in_features=91, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=130, bias=True)
    )
    (batch_norm_layers): ModuleList(
      (0): BatchNorm1d(128